In [ ]:
import cv2
import os
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load the Haar Cascade XML file for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
# Function to capture images and take user input for labels
def capture_images(output_path, num_persons=2, max_images_per_person=5):
    video_capture = cv2.VideoCapture(0)
    os.makedirs(output_path, exist_ok=True)

    all_image_paths = []
    all_labels = []

    for person_id in range(1, num_persons + 1):
        print(f"\nCapturing images for Person {person_id}")
        person_image_paths = []
        person_labels = []

        for image_num in range(1, max_images_per_person + 1):
            ret, frame = video_capture.read()
            cv2.imshow("Captured Image", frame)

            # Take user input for label
            label = input(f"Enter label for Person {person_id}, Image {image_num} or 'exit' to stop: ")

            if label.lower() == 'exit':
                print(f"Exiting capturing for Person {person_id}.")
                break

            image_path = os.path.join(output_path, f"{label}_{person_id}_{image_num}.jpg")
            cv2.imwrite(image_path, frame)

            person_image_paths.append(image_path)
            person_labels.append(label)

        all_image_paths.extend(person_image_paths)
        all_labels.extend(person_labels)

    video_capture.release()
    cv2.destroyAllWindows()

    return all_image_paths, all_labels

In [ ]:
# Function to train the face recognition model
def train_model(csv_filename, image_size=(100, 100), flatten_size=10000):
    df = pd.read_csv(csv_filename)

    images = [cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) for image_path in df["ImagePath"]]
    resized_images = [cv2.resize(image, image_size) for image in images]
    flattened_images = [image.flatten()[:flatten_size] for image in resized_images]

    labels = df["Label"]

    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)

    model = SVC(C=1.0, kernel="linear", gamma="scale")
    model.fit(flattened_images, encoded_labels)

    return model, label_encoder

In [ ]:
# Function to recognize faces in real-time
def recognize_faces(video_capture, trained_model, label_encoder, rectangle_color=(0, 255, 0), label_color=(255, 255, 0)):
    while True:
        ret, frame = video_capture.read()
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face_roi = gray_frame[y:y+h, x:x+w]
            resized_face = cv2.resize(face_roi, (100, 100))
            flattened_face = resized_face.flatten()

            encoded_label = trained_model.predict([flattened_face])[0]
            label = label_encoder.inverse_transform([encoded_label])[0]

            cv2.rectangle(frame, (x, y), (x+w, y+h), rectangle_color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, label_color, 2)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()


In [ ]:
output_path = "newImage"
csv_filename = "face_data.csv"

In [ ]:
# Step 1: Capture images and take user input for labels
image_paths, labels = capture_images(output_path, num_persons=2, max_images_per_person=5)

In [ ]:
# Step 2: Save the captured images and labels to a CSV file
df = pd.DataFrame({"ImagePath": image_paths, "Label": labels})
df.to_csv(csv_filename, index=False)

In [ ]:
# Step 3: Train the model using the captured images
trained_model, label_encoder = train_model(csv_filename)

In [ ]:
# Step 4: Start real-time face recognition using the trained model
video_capture = cv2.VideoCapture(0)
recognize_faces(video_capture, trained_model, label_encoder)